In [5]:
using MeshArrays, FortranFiles

#### Function that reads files that were generated by `trsp_prep`

In [6]:
function trsp_read(mygrid::gcmgrid,dirIn::String)
    #read data from files generated by trsp_prep
    TrspX=read_bin(dirIn*"TrspX.bin",Float32,mygrid)
    TrspY=read_bin(dirIn*"TrspY.bin",Float32,mygrid)
    TauX=read_bin(dirIn*"TauX.bin",Float32,mygrid)
    TauY=read_bin(dirIn*"TauY.bin",Float32,mygrid)
    SSH=read_bin(dirIn*"SSH.bin",Float32,mygrid)

    return TrspX, TrspY, TauX, TauY, SSH
end

trsp_read (generic function with 1 method)

#### Function that generates small binary files (2D) from large netcdf ones (4D)

In [7]:
function trsp_prep(mygrid::gcmgrid,GridVariables::Dict,dirOut::String="")
    
    #wind stress
    fileName="nctiles_climatology/oceTAUX/oceTAUX"
    oceTAUX=Main.read_nctiles(fileName,"oceTAUX",mygrid)
    fileName="nctiles_climatology/oceTAUY/oceTAUY"
    oceTAUY=Main.read_nctiles(fileName,"oceTAUY",mygrid)
    oceTAUX=mask(oceTAUX,0.0)
    oceTAUY=mask(oceTAUY,0.0)

    #sea surface height anomaly
    fileName="nctiles_climatology/ETAN/ETAN"
    ETAN=Main.read_nctiles(fileName,"ETAN",mygrid)
    fileName="nctiles_climatology/sIceLoad/sIceLoad"
    sIceLoad=Main.read_nctiles(fileName,"sIceLoad",mygrid)
    rhoconst=1029.0
    myssh=(ETAN+sIceLoad./rhoconst)
    myssh=mask(myssh,0.0)

    #seawater transports
    fileName="nctiles_climatology/UVELMASS/UVELMASS"
    U=Main.read_nctiles(fileName,"UVELMASS",mygrid)
    fileName="nctiles_climatology/VVELMASS/VVELMASS"
    V=Main.read_nctiles(fileName,"VVELMASS",mygrid)
    U=mask(U,0.0)
    V=mask(V,0.0)
    
    #time averaging and vertical integration
    TrspX=similar(GridVariables["DXC"])
    TrspY=similar(GridVariables["DYC"])
    TauX=similar(GridVariables["DXC"])
    TauY=similar(GridVariables["DYC"])
    SSH=similar(GridVariables["XC"])

    for i=1:mygrid.nFaces
        tmpX=mean(U.f[i],dims=4)
        tmpY=mean(V.f[i],dims=4)
        for k=1:length(GridVariables["RC"])
            tmpX[:,:,k]=tmpX[:,:,k].*GridVariables["DYG"].f[i]
            tmpX[:,:,k]=tmpX[:,:,k].*GridVariables["DRF"][k]
            tmpY[:,:,k]=tmpY[:,:,k].*GridVariables["DXG"].f[i]
            tmpY[:,:,k]=tmpY[:,:,k].*GridVariables["DRF"][k]
        end
        TrspX.f[i]=dropdims(sum(tmpX,dims=3),dims=(3,4))
        TrspY.f[i]=dropdims(sum(tmpY,dims=3),dims=(3,4))
        TauX.f[i]=dropdims(mean(oceTAUX.f[i],dims=3),dims=3)
        TauY.f[i]=dropdims(mean(oceTAUY.f[i],dims=3),dims=3)
        SSH.f[i]=dropdims(mean(myssh.f[i],dims=3),dims=3)
    end

    if !isempty(dirOut)
        write_bin(TrspX,dirOut*"TrspX.bin")
        write_bin(TrspY,dirOut*"TrspY.bin")
        write_bin(TauX,dirOut*"TauX.bin")
        write_bin(TauY,dirOut*"TauY.bin")
        write_bin(SSH,dirOut*"SSH.bin")
    end
    
    return TrspX, TrspY, TauX, TauY, SSH
end    

trsp_prep (generic function with 2 methods)

#### Function that writes a `gcmfaces` object to a binary file using `FortranFile`

In [8]:
function write_bin(inFLD::gcmfaces,filOut::String)
    recl=prod(inFLD.grid.ioSize)*4
    tmp=Float32.(convert2gcmfaces(inFLD))
    println("saving to file: "*filOut)
    f =  FortranFile(filOut,"w",access="direct",recl=recl,convert="big-endian")
    write(f,rec=1,tmp)
    close(f) 
end

write_bin (generic function with 1 method)